In [9]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Conv1D,MaxPooling1D, Flatten

from keras.utils import multi_gpu_model
from keras import backend as K
from sklearn.cross_validation import train_test_split
import matplotlib
matplotlib.use('agg')
import pylab as plt
import pandas as pd
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import Normalizer
from keras.utils import np_utils
import pickle
import h5py

# KDD + and - 
X_testN_ = pd.read_pickle("./dataset/kdd_test__2labels.pkl")
X_trainP_ = pd.read_pickle("./dataset/kdd_train_2labels.pkl")
Y_trainP_ = X_trainP_.iloc[:, 122:].as_matrix()
Y_testN_ = X_testN_.iloc[:, 122:].as_matrix()
X_trainP_ = X_trainP_.as_matrix()
X_testN_ = X_testN_.as_matrix()
scaler1 = Normalizer().fit(X_trainP_[:,:122])
X_trainP_ = scaler1.transform(X_trainP_[:,:122])
scaler2 = Normalizer().fit(X_testN_[:,:122])
X_testN_ = scaler2.transform(X_testN_[:,:122])
X_trainP_ = np.reshape(X_trainP_, (X_trainP_.shape[0],X_trainP_.shape[1],1))
X_testN_ = np.reshape(X_testN_, (X_testN_.shape[0],X_testN_.shape[1],1))
print(Y_testN_.shape)
print(X_testN_.shape)
print(Y_trainP_.shape)
print(X_trainP_.shape)

(11850, 2)
(11850, 122, 1)
(125973, 2)
(125973, 122, 1)


In [10]:
cnn1d_1 = Sequential([Conv1D(64, 3, padding="same",input_shape=(122, 1)),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(2),
    Activation('sigmoid'),
])
print(cnn1d_1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 122, 64)           256       
_________________________________________________________________
activation_4 (Activation)    (None, 122, 64)           0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 61, 64)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3904)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               499840    
_________________________________________________________________
activation_5 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [11]:
#cnn1d_1 = multi_gpu_model(cnn1d_1, gpus=8)
cnn1d_1.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
# train
start_time = time.time()
history = cnn1d_1.fit(X_trainP_, Y_trainP_, batch_size=64, validation_data=(X_testN_, Y_testN_) ,epochs=25)
end_time = time.time() 
print("Total time taken to train the Training model is", (end_time - start_time))
# serialize model to JSON
model_json = cnn1d_1.to_json()
with open("PN.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn1d_1.save_weights("PN.h5")
print("Saved model to disk")
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('Acc_ValaccuaracyPN.png')

Train on 125973 samples, validate on 11850 samples
Epoch 1/25
125973/125973 [==============================] - 30s 239us/step - loss: 0.1447 - acc: 0.9477 - val_loss: 1.6933 - val_acc: 0.5634
Epoch 2/25
125973/125973 [==============================] - 30s 235us/step - loss: 0.0992 - acc: 0.9648 - val_loss: 1.6105 - val_acc: 0.5686
Epoch 3/25
125973/125973 [==============================] - 30s 236us/step - loss: 0.0853 - acc: 0.9678 - val_loss: 2.0379 - val_acc: 0.5569
Epoch 4/25
125973/125973 [==============================] - 31s 246us/step - loss: 0.0748 - acc: 0.9721 - val_loss: 2.0530 - val_acc: 0.5703
Epoch 5/25
125973/125973 [==============================] - 29s 230us/step - loss: 0.0689 - acc: 0.9742 - val_loss: 2.1627 - val_acc: 0.5588
Epoch 6/25
125973/125973 [==============================] - 29s 233us/step - loss: 0.0617 - acc: 0.9767 - val_loss: 1.9887 - val_acc: 0.5922
Epoch 7/25
125973/125973 [==============================] - 30s 240us/step - loss: 0.0574 - acc: 0.9781